# Linear text segmentation

<!-- {{ add_binder_block(page) }} -->

## Introduction

Linear text segmentation is a Natural Language Processing task consisting in dividing a written text into several meaningful segments. Linear text segmentation can be seen as a change point detection task and therefore can be carried out with `ruptures`. 

For our current setup, the atomic unit is the sentence: we are searching for the best set of boundaries for which all sentences between two consecutive boundaries are in the same meaningful segment. We replicate the methodology introduced in [[Choi2000](#Choi2000)]. 

### Data

We use the [[Choi Dataset](#ChoiDataset)]. It is a widely used reference dataset to test whether a segmentation algorithm can separate a written text into several meaningfull topic units. Each data file concatenates the first `n` sentences from ten different documents chosen at random from a 124 documents subset of the [[Brown Corpus](#BrownCorpus)] (the ca\**.pos and cj\**.pos sets). 

In order to evaluate the true power of the Linear Text Segmentation algorithm, `n` is randomly chosen within a range and there are several option offered by the [[Choi Dataset](#ChoiDataset)]. Here is an example of what the data might look like for $n \in [3–5]$

<center>
<img src="../images/choi_example.png" alt="Choi Dataset Snapshot" width="50%"/>
</center>

In the current example, we choose $n \in [9–11]$, meaning the dataset will be the concatenation of 9 to 11 consecutives sentences from ten different documents. 




### Setup

First, we make the necessary imports.

In [ ]:
import numpy as np
from os.path import join  # to access the datafile
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import ruptures as rpt  # our package
from ruptures.base import BaseCost  # To create custom cost
import textwrap  # Format text output nicely

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download("stopwords")
nltk.download("punkt")

import string  # For the ponctuation
import re  # For regular expression, especially to remove the "'s" pattern
from sklearn.feature_extraction.text import (
    CountVectorizer,
)  # For counting word frequency

We also define a utility function.

In [ ]:
def fig_ax(figsize=(15, 5), dpi=150):
    """Return a (matplotlib) figure and ax objects with given size."""
    return plt.subplots(figsize=figsize, dpi=dpi)

## Read and preproccess data

For the preproccessing, we follow the steps described in [[Choi2000](#Choi2000)] section `3.1`, i.e. :

* Remove punctuation and stopwords : `Punctuation and uninformative words are removed from each sentence using a simple regular expression pattern matcher and a stopword list`,
* Tokenize words,
* Stem remaining tokens : `A stemming algorithm (Porter, 1980) is then applied to the remaining tokens to obtain the word stems`. We also use the [[Porter1980](#Porter1980)] algorythm.

In [ ]:
def read_datafile(filename: str):
    """Read the data and apply the preprocessing
    Args:
        filename (str): full path the data file

    Returns:
        original (list): list of sentences, original data without preprocessing applied
        bkps (list): sorted list of breakpoints
        preprocessed (list): list of sentences, once preprocessing has been applied
    """
    original = []
    preprocessed = []
    bkps = []
    with open(filename) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            # Handle ground truth boundaries
            if line == "==========\n":
                # Skip the first boundary in file header
                if len(original) > 0:
                    bkps.append(i - 1 - len(bkps))
                continue
            else:
                original.append(line)

                # Apply preprocessing
                line = re.sub("'s", "", line)  # remove "'s" strings
                line = line.translate(
                    str.maketrans("", "", string.punctuation)
                )  # removes punctuation
                text_tokens = word_tokenize(line)  # tokenize
                ps = PorterStemmer()  # stem
                text_tokens = [ps.stem(word) for word in text_tokens]
                tokens_without_sw = [
                    word for word in text_tokens if not word in stopwords.words()
                ]  # removes stopwords

                # Append new preprocessed sentence
                preprocessed.append(tokens_without_sw)
        return original, bkps, preprocessed

In [ ]:
original, bkps, preprocessed = read_datafile(join("../data/", "0.ref"))
n_sentences = len(preprocessed)
n_bkps = len(bkps) - 1

print(f"There are {n_sentences} sentences to be seperated into {n_bkps + 1} segments")

## Find text segment boundaries

### Define similarity measure

As in [[Choi2000](#Choi2000)] section `3.1`, we use the cosine similarity between two sentences. 

$$ sim(x, y) = \frac{\sum_w f_{x, w} \times f_{y, w}}{\sqrt{\sum_w f_{x, w}^{2} \times \sum_w f_{y, w}^{2}}} $$

Where : 

* $f_{x, w}$ denotes the frequency of word $w$ in sentence $x$

In [ ]:
def cosine_similarity(s1: str, s2: str):
    """Compute cosine similarity given two sentences
    Args:
        s1 (str): first sentence
        s2 (str): second sentence

    Returns:
        float: the cosine distance between s1 and s2
    """
    corpus = [
        " ".join(s1),
        " ".join(s2),
    ]
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus).toarray()
    cosine = np.sum(X[0, :] * X[1, :])
    cosine = cosine / (np.sum(X[0, :] * X[0, :]) * np.sum(X[1, :] * X[1, :])) ** 0.5
    return cosine

### Compute similarity matrix

In [ ]:
# Initialize the matrix
similarities = np.zeros((n_sentences, n_sentences))
# Fill the matrix with similarities
for i in np.arange(n_sentences):
    for j in np.arange(i, n_sentences):
        similarities[i, j] = cosine_similarity(preprocessed[i], preprocessed[j])
        similarities[j, i] = similarities[i, j]

### Display similarity matrix

In [ ]:
fig, ax = fig_ax((5, 5))
plt.imshow(-np.log(similarities), cmap=cm.plasma)
ax.set_title("Cosine similarities matrix", fontsize=10)
ax.set_xlabel("Sentence index", fontsize=8)
ax.set_ylabel("Sentence index", fontsize=8)
plt.show()

We see that some artifacts appear around the similarity matrix diagonal where the cosine measure are a bit higher. 

The similarity matrix seems to be noisy when looking with a naked eye. We will see that this is not an issue for `ruptures` since the cosine similarity approach offers some nice mathematical properties that we describe below. 

### Search best boudaries

Since the cosine similarity is a positive semi-definite kernel $k(\cdot, \cdot) : \mathbb{R}^d\times \mathbb{R}^d \mapsto \mathbb{R}$, with in our case 

$$ k(x, y) = \frac{\langle x\mid y\rangle}{|x||y|} = sim(x, y) $$

Where $\langle \cdot\mid\cdot \rangle$ and $| \cdot |$ are the Euclidean scalar product and norm respectively.

We can then write the cost function as follows :

$$ c(y_{a..b}) = D_{a..b} - \frac{1}{b-a} S_{a..b, a..b} $$

With :

* $y_{a..b}$: written text with sentences with indexes within $[a, b[$
* $c(y_{a..b})$: cost on segment $y_{a..b}$
* $D_{a..b} = \sum_{t=a}^{b-1} k(y_t, y_t)$
* $S_{a..b, a'..b'} = \sum_{a \leq s < b } \sum_{a' \leq t < b'} k(y_s, y_t)$
* $D_{a..a} = 0$
* $S_{a..a, a'..b'} = S_{a..b, a'..a'} = 0$ 

Therefore, based on the computed similarity matrix (which is a Gram matrix), we create a [Custom Cost](../../user-guide/costs/costcustom) as follows. 

In [ ]:
class MyCost(BaseCost):
    """Custom cost for gram matrix."""

    # The 2 following attributes must be specified for compatibility.
    model = ""
    min_size = 2

    def fit(self, gram_matrix):
        """Set the internal parameter."""
        self.signal = gram_matrix
        return self

    def error(self, start, end) -> float:
        """Return the approximation cost on the segment [start:end].
        Args:
            start (int): start of the segment
            end (int): end of the segment
        Returns:
            segment cost
        Raises:
            NotEnoughPoints: when the segment is too short (less than `min_size` samples).
        """
        if end - start < self.min_size:
            raise NotEnoughPoints
        sub_gram = self.signal[start:end, start:end]
        val = np.diagonal(sub_gram).sum()
        val -= sub_gram.sum() / (end - start)
        return val

In [ ]:
algo = rpt.Dynp(custom_cost=MyCost(), min_size=2, jump=1).fit(similarities)
result = algo.predict(n_bkps=n_bkps)

In [ ]:
print(result)
print(bkps)

## Display results

### Display boundaries within the text

In [ ]:
c_real_bkps_idx = 0
c_computed_bkps_icx = 0
line_counter = 1
nb_char = 60
for i, sentence in enumerate(original):
    if i == bkps[c_real_bkps_idx]:
        print(f"\n\t{'='*nb_char}\n")
        c_real_bkps_idx += 1
    if i == result[c_computed_bkps_icx]:
        print(f"\t{'*' *nb_char}")
        c_computed_bkps_icx += 1
    sentence_wrap = textwrap.wrap(
        sentence[:-1], width=nb_char
    )  # removes trailing '\n' for readability purposes
    for j, c_sentence_wrap in enumerate(sentence_wrap):
        print(f"{str(line_counter) + '.' if j == 0 else ''}\t{c_sentence_wrap}")
    line_counter += 1

### Display boundaries on the similarity matrix

In [ ]:
fig, ax = fig_ax((5, 5))
boundaries_width = 0.8
previous_bkp = 0
linestyle = "dashed"
color = "black"
for c_bpks in result:
    c_bpks -= 1
    ax.vlines(
        [c_bpks, previous_bkp],
        previous_bkp,
        c_bpks,
        color=color,
        linestyles=linestyle,
        linewidth=boundaries_width,
    )
    ax.hlines(
        [c_bpks, previous_bkp],
        previous_bkp,
        c_bpks,
        color=color,
        linestyles=linestyle,
        linewidth=boundaries_width,
    )
    previous_bkp = c_bpks
plt.imshow(-np.log(similarities), cmap=cm.plasma)
ax.set_title("Cosine similarities matrix\nWith computed boudaries", fontsize=10)
ax.set_xlabel("Sentence index", fontsize=8)
ax.set_ylabel("Sentence index", fontsize=8)
plt.show()

## Reference

<a id="Choi2000">[Choi2000]</a>
Freddy Y. Y. Choi , NAACL 2000: Proceedings of the 1st North American chapter of the Association for Computational Linguistics conference, April 2000, Pages 26–33, accessible [here](https://www.aclweb.org/anthology/A00-2004.pdf)

<a id="ChoiDataset">[ChoiDataset]</a>
The dataset can be obtained from an archived version of the C99 segmentation [code release](http://web.archive.org/web/20010422042459/http://www.cs.man.ac.uk/~choif/software/C99-1.2-release.tgz). We thank [[Alemi & Ginsparg](#Alemi_Ginsparg)] for pointing to the dataset link. 

<a id="BrownCorpus">[BrownCorpus]</a>
Manual accessible [here](http://icame.uib.no/brown/bcm.html), Henry Kučera and W. Nelson Francis, Brown University, Department of Linguistics, 1964, revised 1971, Revised and Amplified 1979

<a id="Alemi_Ginsparg">[Alemi & Ginsparg]</a>
Alexander A Alemi and Paul Ginsparg, Text Segmentation based on Semantic Word Embeddings, March 15th 2015, Cornell University, accessible [here](https://arxiv.org/pdf/1503.05543.pdf)

<a id="Porter1980">[Porter1980]</a>
M. Porter. 1980. An algorithm for suffix stripping. Program, 14(3):130-137, July. We 

## To be deleted

In [ ]:
# rank, to be deleted
def get_rank(matrix: np.array, vicinity: int):
    res = np.zeros(matrix.shape)
    n_lines, n_columns = matrix.shape
    for i in np.arange(n_lines):
        for j in np.arange(i, n_columns):
            sub_matrix = matrix[
                max(0, i - vicinity) : min(i + vicinity, n_lines),
                max(0, j - vicinity) : min(j + vicinity, n_columns),
            ]
            res[i, j] = np.sum(np.where(sub_matrix < matrix[i, j]))
            res[j, i] = res[i, j]
    return res


similarities_rank = get_rank(similarities, 3)
print(similarities_rank.shape)
fig, ax = fig_ax((5, 3))
plt.imshow(-similarities_rank, cmap=cm.plasma)
plt.show()

In [ ]:
algo = rpt.Pelt(custom_cost=MyCost(), min_size=2, jump=1).fit(similarities)
res_n_bkps = []
res_sum_of_cost = []
x = np.logspace(-0.5, 0.5, num=100)
for pen in x:
    result = algo.predict(pen=pen)
    # print(result)
    res_n_bkps.append(len(result) - 1)
    res_sum_of_cost.append(algo.cost.sum_of_costs(result))

fig, ax = fig_ax((5, 3))
ax.plot(x, res_n_bkps, "b-")
ax.set_xlabel("penality")
ax.set_ylabel("Number of computed break points", color="b")
ax.vlines(x[58], 0, 50, colors="red")

ax2 = ax.twinx()
ax2.plot(x, res_sum_of_cost, "r.")
ax2.set_ylabel("Sum of costs", color="r")

In [ ]:
algo = rpt.Dynp(custom_cost=MyCost(), min_size=2, jump=1).fit(similarities)


def get_sum_of_cost(algo, n_bkps) -> float:
    """Return the sum of costs for the change points `bkps`"""
    bkps = algo.predict(n_bkps=n_bkps)
    return algo.cost.sum_of_costs(bkps)


n_bkps_max = 20  # K_max
array_of_n_bkps = np.arange(1, n_bkps_max + 1)

fig, ax = fig_ax((5, 3))
ax.plot(
    array_of_n_bkps,
    [get_sum_of_cost(algo=algo, n_bkps=i) for i in array_of_n_bkps],
    "-*",
    alpha=0.5,
)
ax.set_xticks(array_of_n_bkps)
ax.set_xlabel("Number of change points")
ax.set_title("Sum of costs")
ax.grid(axis="x")
ax.set_xlim(0, n_bkps_max + 1)